<a href="https://colab.research.google.com/github/akdubey/AKDU/blob/main/Lecture_6d_Data_Aggregation_and_Group_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Aggregation and Group Operations

# What is data aggregation?

* Data aggregation is the process of gathering data and presenting it in a summarized format. 
* The data may be gathered from multiple data sources with the intent of combining these data sources into a summary for data analysis.
* Data aggregation is useful for everything from finance or business strategy decisions to product, pricing, operations, and marketing strategies.


# Example

## Aggregate data in business - 

    Companies often collect data on their online customers and website visitors. The aggregate data would include statistics on customer demographic and behavior metrics, such as average age or number of transactions.



# Pnadas Provide following - 
<pre>
Aggregation	        Description
count()	            Total number of items
first(), last()	    First and last item
mean(), median()	Mean and median
min(), max()	    Minimum and maximum
std(), var()	    Standard deviation and variance
mad()	            Mean absolute deviation
prod()	            Product of all items
sum()	            Sum of all items
</pre>

# GroupBy - split-apply-combine


> The split step involves breaking up and grouping a DataFrame depending on the value of the specified key.

> The apply step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.

> The combine step merges the results of these operations into an output array.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'data1' : np.random.randint(100,200,5),
                   'data2' : np.random.randint(200,300,5)})
df

,key1,data1,data2
0,a,195,243
1,a,194,232
2,b,101,222
3,b,161,245
4,a,167,269


### The most basic split-apply-combine operation can be computed with the groupby() method of DataFrame

In [ ]:
df.groupby('key1')

### Note - 

Notice that what is returned is not a set of DataFrames, but a DataFrameGroupBy object.

It has not actually computed anything yet except for some intermediate data about the group key - 'key1'

We can apply an aggregate to this DataFrameGroupBy object, which will perform the appropriate apply/combine steps to produce the desired result

In [ ]:
df.groupby('key1').sum()

,data1,data2
key1,,
a,556,744
b,262,467


In [ ]:
df.groupby('key1').count()

,data1,data2
key1,,
a,3,3
b,2,2


#  groupedby on multiple keys

In [ ]:
df['key2']=['one', 'two', 'one', 'two', 'one']
df

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


In [ ]:
grouped= df.groupby(['key1','key2'])

In [ ]:
grouped.first()

data1  data2
key1 key2              
a    one     195    243
     two     194    232
b    one     101    222
     two     161    245

# Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data

In [ ]:
for name, data in grouped:
    print(name)
    print()
    print(data)

('a', 'one')

  key1  data1  data2 key2
0    a    195    243  one
4    a    167    269  one
('a', 'two')

  key1  data1  data2 key2
1    a    194    232  two
('b', 'one')

  key1  data1  data2 key2
2    b    101    222  one
('b', 'two')

  key1  data1  data2 key2
3    b    161    245  two


In [ ]:
df

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


In [ ]:
for name,data in df.groupby('key1'):
    print(name)
    print(data)
    print()

a
  key1  data1  data2 key2
0    a    195    243  one
1    a    194    232  two
4    a    167    269  one

b
  key1  data1  data2 key2
2    b    101    222  one
3    b    161    245  two



In [ ]:
df.groupby('key1').last()

,data1,data2,key2
key1,,,
a,167,269,one
b,161,245,two


# Selecting a Column or Subset of Columns

Used especially for large datasets, it may be desirable to aggregate only a few columns

In [ ]:
df

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


In [ ]:
df.groupby('key1')['data1'].first()

key1
a    195
b    101
Name: data1, dtype: int64

# Aggregate, filter, transform, apply

GroupBy objects have aggregate(), filter(), transform(), and apply() methods that efficiently implement a variety of useful operations before combining the grouped data.

# aggregate() method

In [ ]:
df

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


In [ ]:
df.groupby('key1').aggregate([max,min])

data1      data2      key2     
       max  min   max  min  max  min
key1                                
a      195  167   269  232  two  one
b      161  101   245  222  two  one

In [ ]:
df.groupby('key1').aggregate

<bound method DataFrameGroupBy.aggregate of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f8c21a9c3d0>>

We can pass a dictionary mapping column names to operations to be applied on that column.

In [ ]:
df.groupby('key1').aggregate({'data1':max,'data2':min})

,data1,data2
key1,,
a,195,232
b,161,222


In [ ]:
df

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


In [ ]:
def abc(a):
    return a.max()-a.min()

In [ ]:
df.groupby('key1').aggregate(abc)

/usr/local/lib/python3.7/dist-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,28,37
b,60,23


# Filtering

A filtering operation allows you to drop data based on the group properties. 

For example, we might want to keep all groups in which the standard deviation is larger than some critical value:


In [ ]:
df

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


In [ ]:
df.groupby('key1').std()

,data1,data2
key1,,
a,15.885003,19.000000
b,42.426407,16.263456


In [ ]:
func = lambda x:x['data1'].std()>15

df.groupby('key1').filter(func)

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


# Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. 

For such a transformation, the output is the same shape as the input.

A common example is to center the data by subtracting the group-wise mean:


In [ ]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,185.333333,248.0
b,131.000000,233.5


In [ ]:
df.groupby('key1').transform(lambda x: x-x.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  """Entry point for launching an IPython kernel.


,data1,data2
0,9.666667,-5.0
1,8.666667,-16.0
2,-30.000000,-11.5
3,30.000000,11.5
4,-18.333333,21.0


# apply() 

The apply() method lets you apply an arbitrary function to the group results. 

The function should take a DataFrame, and return either a Pandas object (e.g., DataFrame, Series) or a scalar; the combine operation will be tailored to the type of output returned.

For example, here is an apply() that normalizes the first column by the sum of the second:

In [ ]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

In [ ]:
df

,key1,data1,data2,key2
0,a,195,243,one
1,a,194,232,two
2,b,101,222,one
3,b,161,245,two
4,a,167,269,one


In [ ]:
df.groupby('key1').apply(norm_by_data2)

,key1,data1,data2,key2
0,a,0.262097,243,one
1,a,0.260753,232,two
2,b,0.216274,222,one
3,b,0.344754,245,two
4,a,0.224462,269,one


# Reset indices

In [ ]:
g = df.groupby('key1')

In [ ]:
g.sum()

,data1,data2
key1,,
a,556,744
b,262,467


In [ ]:
g.sum().reset_index()

,key1,data1,data2
0,a,556,744
1,b,262,467


# Case Study

### Covid 19

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
import os

In [ ]:
# Mount Your Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
actualdata = '/content/drive/My Drive/Training/covid_19_data.csv'

In [ ]:
# actual_data = pd.read_csv('covid_19_data.csv')

In [ ]:
print(actualdata)

/content/drive/My Drive/Training/covid_19_data.csv


In [ ]:
print(actualdata.shape)

AttributeError: ignored

In [ ]:
actualdata.info()

AttributeError: ignored

In [ ]:
actual_data.isna().sum()

AttributeError: ignored

In [ ]:
clean_data = actual_data.drop(['Province/State'],axis=1)

In [ ]:
clean_data.isna().sum()

NameError: ignored

In [ ]:
clean_data['ObservationDate'] = pd.to_datetime(clean_data['ObservationDate'])

In [ ]:
clean_data.info()

NameError: ignored

In [ ]:
clean_data.head(20)

In [ ]:
case_over_time = clean_data.groupby('ObservationDate')['Confirmed','Deaths','Recovered'].sum()

In [ ]:
case_over_time

NameError: ignored

In [ ]:
import seaborn as sns
case_over_time.plot()

In [ ]:
clean_data.head()

In [ ]:
country = clean_data.groupby('Country/Region')['Confirmed','Deaths','Recovered'].sum()

NameError: ignored

In [ ]:
top5 = country.tail(5)
plt.bar(top5.index,top5['Confirmed'])

In [ ]:
country.loc['India']

NameError: ignored

In [ ]:
country.sort_values('Confirmed',inplace=True)

In [ ]:
country.tail(5)

In [ ]:
top5 = country.tail(5)
plt.bar(top5.index,top5['Confirmed'])

NameError: ignored

In [ ]:
country.sort_values('Deaths',inplace=True)
top5 = country.tail(5)
plt.bar(top5.index,top5['Deaths'])